  -----
<center><font size="18">基于TensorFlow使用Softmax</font></center>    
<center><font size="18">回归模型进行手写体识别</font></center>  
<center><font size="5">平台产品事业本部技术发展部</font></center>
<br/>
<center><font size="4">作者：胡永亮(hu-yl@neusoft.com)</font></center> 

这篇文章主要内容是对TensorFlow官网文章[深度学习方法处理Mnist](https://tensorflow.google.cn/get_started/mnist/pros)的翻译和学习，同时增加了一些个人的理解和相关技术的补充说明，本文结合代码说明了使用Softmax回归模型进行手写体识别。使用卷积神经网络进行手写体识别的部分暂时未包含。由于我也是接触机器学习不久，很多理解不深入或者有错误的地方，请大家指正，互相学习。
### 背景知识
该文章面向的读者，需要了解TensorFlow的基本编程方法、基本的机器学习原理。  
本文先后通过传统的机器学习和卷积神经网络两种算法对手写体数字进行识别。
手写体数据集是28x28的矩阵集合，每个矩阵元素代表图片像素点的颜色深浅。更详细关于数据集的介绍和使用Softmax回归算法进行手写体识别的原理，参考[机器学习初学者处理Mnist](https://tensorflow.google.cn/get_started/mnist/beginners)。  
首先跑了一遍Softmax逻辑回归的机器学习算法，得出准确率是92%。
为了提升准确率，引入深度学习的卷积神经网络算法，得出准确率是99.2%。
### 开发环境准备
我使用了Windows系统，安装了[Anaconda](https://www.anaconda.com/what-is-anaconda/)。它是世界上最流行的python数据科学平台，可以让繁琐的python和tensoflow环境搭建变得触手可得。成功安装Anaconda后，再安装tensorflow，具体操作步骤如下：  
1. 打开Anaconda Prompt
2. 执行conda create -n tensorflow python=3.5，创建一个运行环境，tensorflow是python虚拟环境的名称。
3. 执行activate tensorflow，激活上一个步骤创建的tensorflow运行环境。
4. 使用pip安装，执行pip install tensorflow安装最新的tensorflow版本，我安装的是最新版本1.4。
5. 在notebook中增加刚才新创建的虚拟环境作为新内核。在需要增加的kernel对应的运行环境(比如上面创建的tensorflow)下安装ipykernel，执行pip install ipykernel
6. 在这个环境下输入python -m ipykernel install —name XXXX，后边的名字可以自己定义，用于在notebook里显示。
7. 要想指定notebook的工作目录，在该目录下打开终端，并运行ipython notebook命令即可。

Anaconda安装后自带的交互式开发工具Jupyter notebook在做数据处理方面很方便，推荐使用。本文的代码就是使用notebook开发调试，下载后可以直接使用notebook打开执行。
### 关于数据集
首先下载数据集，由于公司代理的限制，没有使用input_data在线下载。
通过其他方式下载完数据集，放到程序同级目录mnist_dataset下，就可以直接使用了。
需要该数据集文件的同学可以给我发邮件，我通过东软云盘上共享给你，东软云盘只能选择共享人，不能全员共享，有待改进。


In [4]:
import os
from tensorflow.examples.tutorials.mnist import input_data

data_path = '%s%smnist_dataset' % (os.getcwd(), os.sep)
# one_hot变量是指一个特殊的向量，例如，3表示为[0,0,0,1,0,0,0,0,0,0]
mnist = input_data.read_data_sets(data_path, one_hot=True)

Extracting E:\work\code\AI\deeplearning\mnist_dataset\train-images-idx3-ubyte.gz
Extracting E:\work\code\AI\deeplearning\mnist_dataset\train-labels-idx1-ubyte.gz
Extracting E:\work\code\AI\deeplearning\mnist_dataset\t10k-images-idx3-ubyte.gz
Extracting E:\work\code\AI\deeplearning\mnist_dataset\t10k-labels-idx1-ubyte.gz


### Session介绍
Tensorflow的计算能力依赖其高效的C++实现的后端，tensorflow的计算任务通过Session与后台进行连接。TensorFlow的通用方法是，首先创建一个计算流图，
然后使用Session来启动它。
这里，我们不直接使用Session，而是使用InteractiveSession，它使得我们更加灵活地构建代码。它允许你构建新的计算图和已经在运行的计算图进行交互。
尤其你在使用交互式上下文(例如IPthon)时显得更加方便。如果你不使用InteractiveSession，你需要先构建完一整张计算流图，然后开始一个Session去启动
这个计算流图。

In [5]:
import tensorflow as tf
sess = tf.InteractiveSession()

### 计算图
要在Python下进行高效的数学计算，通常我们会使用像NumPy这样的库，把像矩阵乘法这样的耗费计算资源的操作放到Python外执行，以便使用更高效的编程语言实现的代码。即便这样，不幸的是，我们在每个操作切换回Python时依然有很多间接的效率损耗。尤其你在GPU上计算或以分布式方式进行计算，这个间接损耗特别严重，因为这样的情况计算机要花费很高的成本用于数据的传输。
TensorFlow也把大型计算放到Python外执行，并且更一步去避免这个间接的损耗。它不是把单一耗费计算的操作独立放到Python外执行，而是允许我们描述所有关联操作的整张图并放到Python外进行计算。这种方法在Theano和Torch也有用到。
因此，Python代码的作用就是构建一张外部的计算图，然后决策运行计算图的哪个部分。更多细节查看[计算图](https://tensorflow.google.cn/get_started/get_started#the_computational_graph)和[TensorFlow入门](https://tensorflow.google.cn/get_started/get_started)

### 构建Softmax回归模型
在这部分，我们首先构建softmax回归模型的单一层。在下一部分，我们扩展这个为使用softmax回归的多层的卷积网络。
- Placeholders(占位符)  
通过占位符我们开始构建计算图，使用占位符创建图片输入和目标分类输出的结点。
下面代码声明的结点x是一个n行784列的矩阵，每一行代表一张图片(28x28矩阵扁平化)，会在之后运行阶段使用图片训练集进行赋值。
y\_是n行10列的矩阵，每一行是一个one-hot向量，代表输入图片x对应的数字(0-9中的一个)，会在之后运行阶段使用标签赋值。


In [6]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

- 变量  
下面我们定义模型的权重W和偏移量b，W是784行10列的矩阵，b是10维的向量。它们也就是我们求解的模型参数。总的参数个数是：784x10 + 10 = 7850个。它的值可以在计算图运行中使用和被修改。  
tf.zeros(shape)生成一个shape指定形状的全零的矩阵。  
变量在使用之前，需要显示调用tf.global_variables_initializer()进行变量的初始化。

In [7]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

### 预测的分类和损失函数  
- 模型定义  
下面我们就可以定义回归模型了，仅需要一行代码，即使用向量化的图片输入x乘以权重W，加上偏移量b。

In [8]:
y = tf.matmul(x,W) + b

<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>
- 损失函数  
损失函数(Loss or cost function)对假设的函数(定义的模型)进行评价，损失越小的函数，说明对训练数据拟合的越好。通过不断缩小损失函数的值来找到我们期望的最佳模型。
损失函数可以有很多种，比如下面的公式，是方差损失函数：  
![Loss Function](http://img.blog.csdn.net/20161006124407924)  
本文使用了最常用的损失函数-交叉熵。它的思想来自信息论，在多个领域有广泛应用，在机器学习领域，交叉熵可以评估我们的假设函数在描述真实情况的无效性性。要想深入了解交叉熵，请查看这篇[文章](https://colah.github.io/posts/2015-09-Visual-Information)。  它的公式如下：
$$H_{y'}(y) = -\sum_i y'_i \log(y_i)$$

In [9]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

### 模型训练
现在，损失函数有了，模型有了，训练的数据有了，接下来就是要找到模型中参数，使得模型得到预测分类和实际分类差距最小的情况，也就是使得损失函数最小，我们肯定不能逐个去试验W和b的值来找到损失函数最小的情况。  
这时，我们就要使用TensorFlow强大的功能了， TensorFlow内置了很多优化算法，我们可以直接拿内置的优化算法来最小化损失函数，从而找到最佳的权重和偏移量。  
下面使用最常用的优化算法-梯度下降算法，梯度下降的原理：将函数比作一座山，我们站在某个山坡上，往四周看，从哪个方向向下走一小步，能够下降的最快。![示意图](http://img.blog.csdn.net/20161006142959949)
每次前进的一小步，我们称之为学习率(learning rate)，需要调节。下面代码设置的学习率是0.5。

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

上述语句返回一个操作，这个操作将不断地应用梯度下降更新到参数上，因而训练就是反复使用训练数据执行train_step。  
每次取100个训练数据，喂给train_step，前面定义的x是占位符，就是在这里进行数据赋值，其中batch[0]是训练数据的图像输入，batch[1]对应输入的标签。
feed_dict可以替换计算图中的任何张量，不局限于占位符。

### 评估模型
经过1000轮迭代的训练，我们应该可以得到一个不错的模型了。下面我们需要评估一下我们的模型性能如何？是否足够准确地拟合了我们的真实情况。

In [11]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

tf.argmax解释：返回的是索引值，返回每一行或者每一列的最大值的索引，当选择axis=1时，表示每一行的最大值，0表示每列的最大值的索引。
上面语句，也就是返回向量y/y\_的值1所在索引，即代表的数字。  
这给出了一个真假值的列表，为了计算准确率，我们把布尔值转换为浮点数，再求均值，例如[True,True,False,True],转换后[1,1,0,1], 求均值：(1+1+0+1)/4 = 0.75  

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

最后我们使用验证数据集，验证我们模型的精度。

In [13]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9177


### 相关程序和数据集共享说明
我已经把文章对应的notebook开发的程序和数据集放到东软云盘上，有需要的同学可以给我发邮件，我给你增加权限，因为云盘不能全员共享。